<a href="https://colab.research.google.com/github/swha03/G_project/blob/master/cnnmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
import keras
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
import os
import pickle
import cv2
import numpy as np
import pandas as pd
import glob

Using TensorFlow backend.


In [0]:
filepath = '/content/drive/My Drive/Colab Notebooks/sample.dat'
modelpath = '/content/drive/My Drive/Colab Notebooks/mymodel.h5'

In [0]:
def transform_image(image):
    image = image[int(1080 * 0.4):1080, :]
    image = cv2.resize(image, (200, 66))
    return image

In [0]:
def collect_data(dirname, csvname):
    with open(filepath, 'ab') as f:
        data = []
        csv = pd.read_csv(csvname, sep=',')
        joy_values = csv['wheel'].values.tolist()

        images = glob.glob(dirname)

        count = 0

        for img in images:

            screenshot = cv2.imread(img)

            if count < len(joy_values):
                screenshot = transform_image(np.array(screenshot))

                data.append([screenshot, joy_values[count]])

            if count == len(images) - 1:
                print('Collected data count - {0}.'.format(count))
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
                data = [] 

            count += 1

In [0]:
def read_data(split=False):
    with open(filepath, 'rb') as f:
        data = []
        while True:
            try:
                temp = pickle.load(f)

                if type(temp) is not list:
                    temp = np.ndarray.tolist(temp)

                data = data + temp
            except EOFError:
                break
        if split:
            x_train = []
            y_train = []

            for i in range(0, len(data)):
                x_train.append(data[i][0])
                y_train.append(data[i][1])

            return np.array(x_train), np.array(y_train)
        else:
            return np.array(data)

In [0]:
def create_model():
    nrows = 66
    ncols = 200
    img_channels = 3
    output_size = 1

    model = Sequential()
    model.add(Dropout(0.35, input_shape=(nrows, ncols, img_channels)))
    model.add(Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='elu'))
    model.add(Conv2D(filters=36, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='elu'))
    model.add(Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='elu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='elu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='elu'))
    model.add(Dropout(0.35))

    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(output_size))
    model.summary()

    model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(lr=0.001),
                  metrics=['accuracy'])

    return model

In [0]:
def get_model():
    if os.path.isfile(modelpath):
        model = keras.models.load_model(modelpath)
    else:
        model = create_model()

    return model

In [0]:
def train_model(model):
    x, y = read_data(True)

    # test data set 0.15 : training data set 85%
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=1)

    model = get_model()

    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', monitor='val_loss', verbose=1, mode='auto')

    model.fit(x_train, y_train, epochs=8, batch_size=64, verbose=1,
              validation_data=(x_valid, y_valid), callbacks=[checkpoint])

    model.save(modelpath)

In [0]:
def display_data():
    data = read_data()
    data_size = len(data)
    print('Data size  -  ' + str(data_size))

    for i in range(data_size - 1, 0, -1):
        print(str(image[1]))
        cv2.waitKey(50)

In [0]:
collect_data('/content/drive/My Drive/Colab Notebooks/CFile1/*.jpg', '/content/drive/My Drive/Colab Notebooks/CFile1/1.csv')

KeyboardInterrupt: ignored

In [12]:
model = get_model()
train_model(model)

Train on 697 samples, validate on 123 samples
Epoch 1/8
697/697 [==============================] - 5s 8ms/step - loss: 0.0195 - accuracy: 0.0000e+00 - val_loss: 0.0179 - val_accuracy: 0.0000e+00

Epoch 00001: saving model to model-001.h5
Epoch 2/8
697/697 [==============================] - 5s 7ms/step - loss: 0.0194 - accuracy: 0.0000e+00 - val_loss: 0.0179 - val_accuracy: 0.0000e+00

Epoch 00002: saving model to model-002.h5
Epoch 3/8
697/697 [==============================] - 5s 7ms/step - loss: 0.0194 - accuracy: 0.0000e+00 - val_loss: 0.0178 - val_accuracy: 0.0000e+00

Epoch 00003: saving model to model-003.h5
Epoch 4/8
697/697 [==============================] - 5s 7ms/step - loss: 0.0193 - accuracy: 0.0000e+00 - val_loss: 0.0178 - val_accuracy: 0.0000e+00

Epoch 00004: saving model to model-004.h5
Epoch 5/8
697/697 [==============================] - 5s 7ms/step - loss: 0.0193 - accuracy: 0.0000e+00 - val_loss: 0.0177 - val_accuracy: 0.0000e+00

Epoch 00005: saving model to model-0